# Creating a new Measurement Protocol

This notebook is helping you to create a new protocol with an analysis function for the MultispeQ as well as with the needed output for a later use with [openJII](https://openjii.org)

If you haven't done so before, you need to install the following dependencies:

This first library allows you to take measurements with the MultispeQ using your python Notebook (see [Documentation](https://jan-ingenhousz-institute.github.io/JII-MultispeQ/)).

The next library provides functionality around MultispeQ protocols (see [Documentation](https://jan-ingenhousz-institute.github.io/JII-MultispeQ-Protocols/)).

For plotting data, please also make sure to have Matplotlib installed (see [Documentation](https://matplotlib.org/)).

```sh
pip install git+https://github.com/Jan-IngenHousz-Institute/JII-MultispeQ.git --upgrade --no-cache-dir
pip install git+https://github.com/Jan-IngenHousz-Institute/JII-MultispeQ-Protocols.git --upgrade --no-cache-dir
pip install matplotlib
```

In [ ]:
## The following dependencies are imported to use the MultispeQ and develop a protocol
## and analysis function. They are NOT required for the protocol and analysis function
## themselves.
##
## If you use dependencies or functions that are already included here, please import them
## below again to provide a better overview on what dependencies are required by your
## analysis function.

import jii_multispeq.measurement as _measurement
import jii_multispeq.device as _device

from jii_multispeq_protocols import validate as _validate

import matplotlib.pyplot as plt
import json
from string import Template
import inspect

PROTOCOL_CONTENT = f"""
\"\"\"
$description
\"\"\"

_protocol = $protocol

$analyze

_example = $example
"""

## PLEASE DO NOT CHANGE THIS CODE

## Your Protocol Code

MultispeQ Protocols are formatted as a JSON (JavaScript Object Notation). A pyton dictionary can be converted to a JSON, but it cannot contain functions as the MultispeQ is not able to interpret those.

### Protocol Variable

The next cell defines the variable `_protocol` which holds the protocol code. You can create the protocol simply writing the json code, but you can also use functions to build the code, as long as the final output for `_protocol` is a valid JSON.

### Protocol Validation

The cell after that validates the protocol, using the protocol validator function. This will help to identify issues with the protocol structure as well as with value ranges and parameters.

In [ ]:
## Protocol Code

_protocol = [{}]

In [ ]:
## Protocol Validation

is_valid, errors = _validate(_protocol)

if not is_valid:
  for e in errors:
    print(e)
else:
  print("The protocol seems valid, you are good to go!")

## Test your Protocol

Now it is time to test the protocol using the MultispeQ. Run the next cells to connect the MultispeQ instrument and run the previously built protocol. The output of the MultispeQ is saved to the variable `_data`. Further, also a checksum is returned to `_checksum`, but it can be ignored for now.

In [ ]:
## Identify the Port your MultispeQ instrument is connected to
_device.get_ports()

In [ ]:
## Connect your Instrument
_connection = _device.connect('/dev/cu.usbmodem42949672951')

In [ ]:
## Take a Measurement with your protocol
if _device.is_connected(_connection):
  _data, _checksum = _measurement.measure( _connection, _protocol )

In [ ]:
## When done, continue here and disconnect the instrument
_device.disconnect(_connection)

## Your Measurement Analysis

The MultispeQ's output after running the protocol needs to be 

In [ ]:
## Analysis Function Dependencies


## Analysis Function

def _analyze( input ):
  return input

## Test Analysis Function

In [ ]:
## Run the analysis function

_data_analyzed = _measurement.analyze( _data, _analyze )

In [ ]:
## View the output

_measurement.view(_data_analyzed)

## Generate Graphs for data in lists (arrays in JavaScript)

for key, value in _data_analyzed.items():

    # Check if this value is a list
    if not isinstance(value, list) or (len(value) == 0) or not isinstance(value[0], (int, float, list)):
      continue

    # Create the plot
    plt.figure(figsize=(10, 5))
    
    # Add trace / traces
    if (len(value) > 0) and isinstance(value[0], list):
        for idx, val in enumerate(value):
            plt.plot(val, linewidth=2, label="%s #%s" % (key, int(idx)+1))
    elif (len(value) > 0) and isinstance(value[0], (int, float)) :
        plt.plot(value, linewidth=2, label=key)

    # Customize the plot
    plt.grid(True, linestyle='-', axis='y', alpha=0.3)
    plt.xlabel('Pulses', fontdict={"size": 15})
    plt.ylabel("a. u. (%s)" % key, fontdict={"size": 15})
    for spine in ['top', 'right']:
        plt.gca().spines[spine].set_visible(False)
    
    # Show Legend
    plt.legend()

    # Use a tight layout
    plt.tight_layout()

    # Show the plot
    plt.show()

## Add your Protocol and Analysis Function to openJII

Currently you cannot save the protocol and analysis function from the notebook directly.

+ **Protocol:** Run the cell below to generate the protocol output that you then copy and paste to the openJII platform
+ **Analysis Function:** Copy the cell with the function and depenencies above and paste to the openJII platform

In [ ]:
## Copy the output below and add it to openJII as a protocol

_protocol_str = json.dumps(_protocol, separators=(',', ':'))
print(_protocol_str)

## Save as File for Local Use

In [ ]:
## Save Protocol as a Python File

FILENAME = "MyProtocol.py"

DESCRIPTION = """
My Protocol
===========

Your Description goes here
"""

if _protocol and _analyze and _data:
  with open(FILENAME, "w") as file:
    file.write(Template(PROTOCOL_CONTENT).substitute(
      description=DESCRIPTION,
      protocol=json.dumps(_protocol, indent=2),
      analyze=inspect.getsource(_analyze),
      example=json.dumps(_data, indent=2)
    ))